In [6]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
import pandas as pd
import data_pecarn 
import data_psrc
import data
from collections import Counter

# plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2
NUM_PATIENTS = 12044

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
df_pecarn = data_pecarn.get_data(use_processed=False,
                                 frac_missing_allowed=1.0)
all_feats_pecarn, filtered_feats_pecarn = data.get_feat_names(df_pecarn)
df_psrc = data_psrc.get_data(use_processed=False)
all_feats_psrc, filtered_feats_psrc = data.get_feat_names(df_psrc)

computing pecarn preprocessing...
computing psrc preprocessing...


In [174]:
(df_psrc['Complainabd. pain']!='0').astype(int)

0       0
1       0
2       0
3       1
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      1
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      1
22      1
23      1
24      0
25      0
26      1
27      0
28      1
29      0
       ..
2158    0
2159    1
2160    0
2161    1
2162    0
2163    0
2164    0
2165    0
2166    0
2167    0
2168    1
2169    1
2170    0
2171    1
2172    0
2173    0
2174    0
2175    0
2176    0
2177    1
2178    0
2179    0
2180    0
2181    0
2182    0
2183    0
2184    0
2185    1
2186    0
2187    0
Name: Complainabd. pain, Length: 2188, dtype: int64

In [176]:
filtered_feats_psrc

{'AbdDistention_no',
 'AbdDistention_unknown',
 'AbdDistention_yes',
 'AbdTenderDegree_Mild',
 'AbdTenderDegree_Moderate',
 'AbdTenderDegree_Severe',
 'AbdTenderDegree_unknown',
 'AbdTrauma_no',
 'AbdTrauma_yes',
 'AbdomenPain_no',
 'AbdomenPain_yes',
 'Age',
 'DecrBreathSound_no',
 'DecrBreathSound_yes',
 'GCSScore',
 'LtCostalTender',
 'RecodedMOI_Bike collision/fall',
 'RecodedMOI_Fall from an elevation',
 'RecodedMOI_Motor vehicle collision',
 'RecodedMOI_Motorcycle/ATV/Scooter collision',
 'RecodedMOI_Object struck abdomen',
 'RecodedMOI_Pedestrian/bicyclist struck by moving vehicle',
 'RecodedMOI_unknown',
 'RtCostalTender',
 'SeatBeltSign_no',
 'SeatBeltSign_yes',
 'ThoracicTrauma_no',
 'ThoracicTrauma_yes',
 'VomitWretch_no',
 'VomitWretch_unknown',
 'VomitWretch_yes'}

In [177]:
df_psrc['Com']

KeyError: 'Com'

**look at resulting feats**

In [161]:
filtered_feats_pecarn.intersection(filtered_feats_psrc)

{'AbdDistention_no',
 'AbdDistention_unknown',
 'AbdDistention_yes',
 'AbdTenderDegree_Mild',
 'AbdTenderDegree_Moderate',
 'AbdTenderDegree_Severe',
 'AbdTenderDegree_unknown',
 'AbdTrauma_no',
 'AbdTrauma_yes',
 'AbdomenPain_other',
 'Age',
 'DecrBreathSound_no',
 'DecrBreathSound_yes',
 'GCSScore',
 'LtCostalTender',
 'RecodedMOI_Bike collision/fall',
 'RecodedMOI_Fall from an elevation',
 'RecodedMOI_Motor vehicle collision',
 'RecodedMOI_Motorcycle/ATV/Scooter collision',
 'RecodedMOI_Object struck abdomen',
 'RecodedMOI_Pedestrian/bicyclist struck by moving vehicle',
 'RecodedMOI_unknown',
 'RtCostalTender',
 'SeatBeltSign_no',
 'SeatBeltSign_yes',
 'ThoracicTrauma_no',
 'ThoracicTrauma_yes',
 'VomitWretch_no',
 'VomitWretch_unknown',
 'VomitWretch_yes'}

In [162]:
print('psrc but not pecarn')
filtered_feats_psrc.difference(filtered_feats_pecarn)

psrc but not pecarn


set()

In [163]:
print('pecarn but not psrc')
filtered_feats_pecarn.difference(filtered_feats_psrc)

pecarn but not psrc


{'AbdTrauma_or_SeatBeltSign_False',
 'AbdTrauma_or_SeatBeltSign_True',
 'AbdTrauma_unknown',
 'AbdomenPain_no',
 'AbdomenPain_unknown',
 'AbdomenPain_yes',
 'CostalTender',
 'DecrBreathSound_unknown',
 'DistractingPain_no',
 'DistractingPain_unknown',
 'DistractingPain_yes',
 'RecodedMOI_Fall down stairs',
 'SeatBeltSign_unknown',
 'ThoracicTender_no',
 'ThoracicTender_unknown',
 'ThoracicTender_yes',
 'ThoracicTrauma_unknown'}

# write csvs

In [23]:
df_pecarn.astype('object').describe().transpose().to_csv('processed/pecarn_vars.csv')
df_psrc.astype('object').describe().transpose().to_csv('processed/psrc_vars.csv')

In [117]:
common_feats = [x for x in feat_names_pecarn if x in feat_names_psrc]

In [17]:
df_psrc['Emesis post injury'].unique()

array([ 0., nan,  1.])